# Indexing Arrays

For the most part, indexing an MLX `array` works the same as indexing a NumPy `numpy.ndarray`. For example

In [2]:
import mlx.core as mx

arr = mx.arange(10)
arr[3]

array(3, dtype=int32)

In [4]:
arr[-2]  # negative index

array(8, dtype=int32)

In [5]:
arr[2:8:2] # start at 2, stop at 8, step by 2

array([2, 4, 6], dtype=int32)

For multi-dimensional arrays, the `...` or `Ellipsis` syntax works as in NumPy:

In [6]:
arr = mx.arange(8).reshape(2,2,2)
arr[:,:,0]

array([[0, 2],
       [4, 6]], dtype=int32)

In [7]:
arr[...,0]  # same as arr[:,:,0]

array([[0, 2],
       [4, 6]], dtype=int32)

You can index with `None` to create a new axis

In [9]:
arr = mx.arange(10)
arr.shape



(10,)

In [10]:
arr[None].shape

(1, 10)

You can also use an `array` to index another `array`

In [11]:
arr = mx.arange(10)
idx = mx.array([1,3,4])
arr[idx]

array([1, 3, 4], dtype=int32)

## Difference from NumPy

MLX indexing is different from NumPy indexing in two importatn ways:

1. indexing does not perform bounds checking. Indexing out of bounds is undefined behavior.
2. boolean mask based indexing is not yet supported

The reason for the lack of bounds checking is that exceptions cannot propagate from the GPU. Peforming bounds checking for array indices before launching the kernel would be extremely inefficient.

Indexing with boolean masks is something that MLX may supoort in the future. In general, MLX has limited support for operations for which outputs shapes are dependent on the input data. Other examples of these types of operations which MLX does not yet support include `numpy.nonzero()` and the single input version of `numpy.where()`

## In Place Updates

In place updates to indexed arrays are possible in MLX. For example:

In [12]:
a = mx.array([1,2,3])
a[2] = 0
a

array([1, 2, 0], dtype=int32)

Just as in NumPy, in place updates will be reflected in all references to the same array:

In [13]:
a = mx.array([1,2,3])
b = a
b[2] = 0
b

array([1, 2, 0], dtype=int32)

In [14]:
a

array([1, 2, 0], dtype=int32)

Transformations of functions which use in-place updates are allowed and work as expected. For example:

In [15]:
def fun(x,idx):
    x[idx] = 2.0
    return x.sum()


dfdx = mx.grad(fun)(mx.array([1.0,2.0,3.0]),mx.array([1]))

print(dfdx) # [1. 0. 1.]

array([1, 0, 1], dtype=float32)
